In [1]:
# importing required libraries

import numpy as np
from datetime import datetime
import pandas as pd
# import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight

In [2]:
# !pip install scikit-learn

In [8]:
print("TF version: ", tf.__version__)
print("GPU is ", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF version:  2.10.0
GPU is  not available


In [3]:
# Data Paths
train_dir = r"D:\deeplearning\image_cls\image_data_train"
test_dir = r"D:\deeplearning\image_cls\image_data_test"
valid_dir = r"D:\deeplearning\image_cls\image_data_validation"

# Checkpoint Paths
checkpoint_1_path = "checkpoint_1"
checkpoint_2_path = "checkpoint_2"

# Models Paths
model_1_path = "model"
model_path = "model_complete"

# Set hyperparameters
NUM_CLASSES = 6
INPUT_SHAPE = (299, 299, 3)
TARGET_SIZE = (299, 299)
BATCH_SIZE = 64
EPOCHS = 5

In [4]:
# Image Augmentation

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = valid_datagen.flow_from_directory(
    test_dir,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 657 images belonging to 6 classes.
Found 121 images belonging to 6 classes.
Found 121 images belonging to 6 classes.


In [5]:
class_indices = train_generator.class_indices

In [6]:
class_indices

{'explicit_moderation': 0,
 'gore_moderation': 1,
 'intoxicants_moderation': 2,
 'sfw_moderation': 3,
 'suggestive_moderation': 4,
 'weapons_moderation': 5}

In [7]:
# Defining steps per epoch

TRAIN_STEPS = train_generator.n // BATCH_SIZE
VALID_STEPS = validation_generator.n // BATCH_SIZE

In [8]:
# Balancing the classes

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights = {i:w for i,w in enumerate(class_weights)}

In [9]:
from tensorflow.keras.models import Model

# Loading the pre-trained model
base_model = Xception(weights='imagenet', include_top=True, input_shape=INPUT_SHAPE)
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [10]:
  
base_model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
base_model.trainable = False
base_model.summary()
model = tf.keras.Sequential([
    base_model,
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(rate=0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
# Compile Model
model.compile(optimizer=Adam(),
                       loss="categorical_crossentropy",
                       metrics=["accuracy", "CategoricalAccuracy", "Precision", "Recall"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              20861480  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 21,124,526
Trainable params: 263,046
Non-trainable params: 20,861,480
_________________________________________________________________


In [12]:
# Callbacks

checkpoint_cb = ModelCheckpoint(filepath=checkpoint_1_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

In [13]:

from datetime import datetime
from PIL import Image


try:
    # Training the model
    start = datetime.now()

    history = model.fit(
        train_generator,
        steps_per_epoch=TRAIN_STEPS,
        validation_data=validation_generator,
        validation_steps=VALID_STEPS,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=[checkpoint_cb])

    end = datetime.now()
    print(f"Execution time: {(end-start)}")

    # saving the model
    model.save(model_1_path, save_format="tf")

except PIL.UnidentifiedImageError as e:
    print("Error: UnidentifiedImageError")
    # Handle the exception here, e.g., print the problematic image file name, skip it, or take any other appropriate action.
    # For example, you can inspect the image file path by accessing `e.args[0]`.
except Exception as e:
    # Handle any other unexpected exceptions here, if needed.
    print(f"An error occurred: {e}")


Epoch 1/5
10/10 [==============================] - 48s 5s/step - loss: 1.5646 - accuracy: 0.4013 - categorical_accuracy: 0.4013 - precision: 0.7778 - recall: 0.0590 - val_loss: 1.4943 - val_accuracy: 0.5938 - val_categorical_accuracy: 0.5938 - val_precision: 1.0000 - val_recall: 0.0156
Epoch 2/5
10/10 [==============================] - 46s 5s/step - loss: 1.0415 - accuracy: 0.6543 - categorical_accuracy: 0.6543 - precision: 0.9144 - recall: 0.3423 - val_loss: 1.1971 - val_accuracy: 0.4688 - val_categorical_accuracy: 0.4688 - val_precision: 0.7917 - val_recall: 0.2969
Epoch 3/5
10/10 [==============================] - 45s 4s/step - loss: 0.7178 - accuracy: 0.7841 - categorical_accuracy: 0.7841 - precision: 0.8977 - recall: 0.5919 - val_loss: 1.1844 - val_accuracy: 0.5781 - val_categorical_accuracy: 0.5781 - val_precision: 0.6579 - val_recall: 0.3906
Epoch 4/5
10/10 [==============================] - 45s 4s/step - loss: 0.6306 - accuracy: 0.7841 - categorical_accuracy: 0.7841 - precision

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [27]:
# Evaluating the model on test set

metrics = model.evaluate(test_generator)
print(len(metrics))
metrics

2/2 [==============================] - 8s 4s/step - loss: 1.3276 - accuracy: 0.5537 - categorical_accuracy: 0.5537 - precision: 0.7200 - recall: 0.4463
5


[1.3275847434997559,
 0.5537189841270447,
 0.5537189841270447,
 0.7200000286102295,
 0.44628098607063293]

In [20]:
from PIL import Image
import tensorflow as tf
import numpy as np

model_path = "D:\deeplearning\image_cls\model"
model = tf.keras.models.load_model(model_path)

class_names = ['explicit_moderation', 'gore_moderation', 'intoxicants_moderation', 'sfw_moderation', 'suggestive_moderation', 'weapons_moderation']

def get_img_from_local(image_path, img_type='other'):
    image = Image.open(image_path)
    image = image.resize((299, 299))
    return image

def get_predictions(image_path_lst):
    pred_lst = []
    category = []
    for image_path in image_path_lst:
        prediction_dict = {}
        # Loading image
        try:
            image = get_img_from_local(image_path)
        except Exception as e:
            prediction_dict["image_path"] = image_path
            prediction_dict["class_label"] = None
            prediction_dict["confidence"] = None
            prediction_dict["error"] = str(e)
            pred_lst.append(prediction_dict)
            continue
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = image / 255.0  # Normalize the image
        # Expand dimensions to match the input tensor shape
        image = np.expand_dims(image, axis=0)
        # Make predictions
        predictions = model.predict(image)
        # Process the predictions
        predicted_class_index = np.argmax(predictions[0])
        prediction_dict["image_path"] = image_path
        prediction_dict["class_label"] = class_names[predicted_class_index]
        category.append(class_names[predicted_class_index])
        prediction_dict["confidence"] = float(predictions[0][predicted_class_index])
        pred_lst.append(prediction_dict)
    category = list(filter(lambda x: x is not None, category))
    if len(category) == 0:
        final_output = "No images found"
    elif any(element != 'sfw_moderation' for element in category):
        final_output = "spam"
    else:
        final_output = "ham"
    # pred_lst.append({'final_output':final_output})
    # return pred_lst
    return pred_lst, final_output

image_path_lst = ["D:\deeplearning\image_cls\image_data_validation\gore_moderation\gettyimages-498084805-612x612.jpg"]

predictions, final_output = get_predictions(image_path_lst)
print(predictions)
print("Final Output:", final_output)


1/1 [==============================] - 1s 537ms/step
[{'image_path': 'D:\\deeplearning\\image_cls\\image_data_validation\\gore_moderation\\gettyimages-498084805-612x612.jpg', 'class_label': 'gore_moderation', 'confidence': 0.8929468393325806}]
Final Output: spam
